# Project 4: Predicting Bike Share Rides

## Alexander, Michael; Birnbaum, Kevin; Manzano, Romulo

In [8]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
import datetime as dt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot as plot_off
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split


To make sure we understand how our models are performing, we defined a function that helps us understand the squared log error, which is the same metric used by Kaggle on this competition

In [9]:
def rmsle(actual, predicted):
    """
    Computes the squared log error.
    This function computes the squared log error between two numbers,
    or for element between a pair of lists or numpy arrays.
    Parameters
    ----------
    actual : int, float, list of numbers, numpy array
             The ground truth value
    predicted : same type as actual
                The predicted value
    Returns
    -------
    score : double or list of doubles
            The squared log error between actual and predicted
    """
    sle = (np.power(np.log(np.array(actual) + 1) - np.log(np.array(predicted) + 1), 2))
    msle = np.mean(sle)
    rmsle = np.sqrt(msle)
    return rmsle


First we initialize a couple of variables to indicate where we'll be sourcing our data from

In [10]:
## Actual problem training starts here
t_file = 'train.csv'
test_file = 'test.csv'

We proceed by loading the training data, and spliting the features and labels into two dataframes

In [102]:
raw_data = pd.read_csv(t_file)
#specifying the target column names
target_columns = ['count','registered','casual']
regress = ['count']
regress_registered = ['registered']
regress_casual = ['casual']
#labels for the single measure (count) for which score is calculated
labels = raw_data[regress]
all_target_cols = raw_data[target_columns]
data = raw_data.drop(target_columns,axis=1)

In [51]:
#needed for future experimentation
def keep_non_target_cols_only(d,cols = target_columns):
    d = d.drop(cols,axis=1)
    return d

By exploring the volume of bike rentals by hour we infer that subcomponents of the date feature might be more relevant for the accurate modeling of the regressed variable. Here is a sample visualization of the total count for each season.  

In [87]:
%%HTML
<div class='tableauPlaceholder' id='viz1492964583978' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;TimeandSeason&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='KagglebikeSharingDraft&#47;TimeandSeason' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;TimeandSeason&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1492964583978');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [6]:
%%HTML
<div class='tableauPlaceholder' id='viz1492964676437' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;WeatherConditions&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='KagglebikeSharingDraft&#47;WeatherConditions' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;WeatherConditions&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1492964676437');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

The below function helps us xtract the relevant features from the data field

In [20]:
def convert_time(x):
    t = dt.datetime.strptime(x,"%m/%d/%Y %H:%M")
    return pd.Series([t.weekday(),t.hour,t.year,t.month,t])

In [59]:
def feature_engineering_v1(z):
    z[['weekday','hr','yr','month','time']]  = z.apply(lambda x: convert_time(x['datetime']),axis=1)
    #don't need datetime anymore
    z.drop('datetime',axis=1,inplace = True)    
    #don't need at this point
    z.drop('time',axis=1,inplace = True)    
    return z

In [60]:
data = feature_engineering_v1(data)

Some conversion is needed to ensure the data is usable by SK Learn

Train/Test split

In [25]:
train_data,test_data = data[:6000],data[6000:]
train_labels, test_labels = labels[:6000], labels[6000:]

## Adaboost

First we wanted to leverage the simplicity of decision trees to obtained a more optimal performance

In [27]:
#creating an adaBoostRegressor on this
ada = AdaBoostRegressor()
ada_1 = ada.fit(train_data,train_labels)

C:\Users\romulo\AppData\Roaming\Python\Python35\site-packages\sklearn\utils\validation.py:515: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Now we test its accuracy on the test data:

In [29]:
ada_predict = ada_1.predict(test_data)
test_labels_np = np.array(test_labels['count'])
ada_predict_int = (np.array(ada_predict)).astype('int')
ada_performance =  rmsle(test_labels_np,ada_predict_int)
print('RMSLE for adaboost',ada_performance)

RMSLE for adaboost 0.704346653717


Much better than the 1.26 from the original submission!

## Random Forest

We also wanted to test random forest as an alternative given it hedges against overfitting

In [30]:
#Using 1k estimators
forest = RandomForestRegressor(n_estimators = 1000, n_jobs = -1)

forest_1 = forest.fit(train_data,train_labels)
forest_1_predict = forest_1.predict(test_data)
forest_1_predict_int = (np.array(forest_1_predict)).astype('int')
forest_1_performance =  rmsle(test_labels_np,forest_1_predict_int)
print('RMSLE with random forest:', forest_1_performance)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSLE with random forest: 0.459296236342


As we can observe we obtained better results by using an approach less sensitive to noise in training data (overfitting)

## Next Steps?

- Binarizing variables such as Season, workingdays, weather and hours might yield improved results.
- Creating an ensemble of predictors based on say (a) a model that predicts registered user count (b) a model that predicts casual user count and (c) a model that uses a & b and additional features as inputs to predict total count

## Binarizing some of the variables

Let's begin by defining some of the variables we might want to convert from numerical (could be interpreted as ordinal) to dummy/binary standalone features

We'll begin with Season and Weather

In [31]:
binarize_cols = ['season','weather']

In [32]:
data_bin = pd.get_dummies(data,columns = binarize_cols )

In [33]:
train_data_bin,test_data_bin = data_bin[:6000],data_bin[6000:]
train_labels_bin, test_labels_bin = labels[:6000], labels[6000:]

In [34]:
#Using 1k estimators
forest_bin = RandomForestRegressor(n_estimators = 1000, n_jobs = -1)

forest_2 = forest_bin.fit(train_data_bin,train_labels_bin)
forest_2_predict = forest_2.predict(test_data_bin)
forest_2_predict_int = (np.array(forest_2_predict)).astype('int')
forest_2_performance =  rmsle(test_labels_np,forest_2_predict_int)
print('RMSLE with random forest w dummies:', forest_2_performance)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSLE with random forest w dummies: 0.458992381572


De minimis effect!

## Random Train and Test split

In [35]:
train_data_rm,test_data_rm,train_labels_rm ,test_labels_rm = train_test_split(data_bin, labels, test_size=0.20, random_state=42)

In [37]:
test_labels_random_np = np.array(test_labels_rm['count'])

forest_3_rm = RandomForestRegressor(n_estimators = 1000, n_jobs = -1)

forest_3 = forest_3_rm.fit(train_data_rm,train_labels_rm)
forest_3_predict = forest_3.predict(test_data_rm)
forest_3_predict_int = (np.array(forest_3_predict)).astype('int')
forest_3_performance =  rmsle(test_labels_random_np,forest_3_predict_int)
print('RMSLE with random forest w dummies:', forest_3_performance)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSLE with random forest w dummies: 0.323865662722


## Feature Engineering

**Popularity**

In [4]:
%%HTML
<div class='tableauPlaceholder' id='viz1492964529434' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;PopularityEffect&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='KagglebikeSharingDraft&#47;PopularityEffect' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ka&#47;KagglebikeSharingDraft&#47;PopularityEffect&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1492964529434');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Time on market** will serve as an proxy for popularity. We will use the number of weeks since first measure is available (we'll call that launch date!)

In [40]:
def time_on_market(x,begining):
    return (x-begining).days // 7

In [43]:
def get_earliest_date(z):
    z[['weekday','hr','yr','month','time']]  = z.apply(lambda x: convert_time(x['datetime']),axis=1)
    #don't need datetime anymore
    return min(z['time'])    

In [76]:
def feature_engineering_v2(z,market_launch):
    z[['weekday','hr','yr','month','time']]  = z.apply(lambda x: convert_time(x['datetime']),axis=1)
    #don't need datetime anymore
    z.drop('datetime',axis=1,inplace = True)
    #calculate market launch
    z['time_on_market']  = z.apply(lambda x: time_on_market(x['time'],market_launch),axis=1)
    z.drop('time',axis=1,inplace=True)
    z = pd.get_dummies(z,columns = binarize_cols )
    z.drop('yr',axis=1,inplace = True)
    return z

Defining baseline data again

In [77]:
data_2 = keep_non_target_cols_only(raw_data)

In [62]:
market_launch = get_earliest_date(data_2)
market_launch

Timestamp('2011-01-01 00:00:00')

In [78]:
data_2 = feature_engineering_v2(data_2,market_launch)

In [79]:
train_data_v2,test_data_v2,train_labels_v2 ,test_labels_v2 = train_test_split(data_2, labels, test_size=0.20, random_state=42)

In [80]:
test_labels_v2_np = np.array(test_labels_v2['count'])

forest_4_v2 = RandomForestRegressor(n_estimators = 1000, n_jobs = -1)

forest_4 = forest_4_v2.fit(train_data_v2,train_labels_v2)
forest_4_predict = forest_4.predict(test_data_v2)
forest_4_predict_int = (np.array(forest_4_predict)).astype('int')
forest_4_performance =  rmsle(test_labels_v2_np,forest_4_predict_int)
print('RMSLE with random forest w dummies & time on market:', forest_4_performance)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSLE with random forest w dummies & time on market: 0.320638578487


Minimal improvement

## Rush Hours and Midnight!

In [81]:
def feature_engineering_v3(z,market_launch):
    z = feature_engineering_v2(z,market_launch)
    z['morning_rush'] = ((z['hr']>=7)  & (z['hr']<=9 ) & (z['workingday']==1 ) ) *1
    z['evening_rush'] = ((z['hr']>=16)  & (z['hr']<=19 ) & (z['workingday']==1 )) *1
    z['sleep_time'] = (((z['hr']>=22)  | (z['hr']<=5 )) & (z['workingday']==1 )) *1
    z['weekend_rush'] = ((z['hr']>=11)  | (z['hr']<=18 ) & (z['weekday']>=5 )) *1
    return z

In [82]:
data_3 = keep_non_target_cols_only(raw_data)
data_3 = feature_engineering_v3(data_3,market_launch)
train_data_v3,test_data_v3,train_labels_v3 ,test_labels_v3 = train_test_split(data_3, labels, test_size=0.20, random_state=42)

In [83]:
test_labels_v3_np = np.array(test_labels_v3['count'])

forest_5_v3 = RandomForestRegressor(n_estimators = 1000, n_jobs = -1)

forest_5 = forest_5_v3.fit(train_data_v3,train_labels_v3)
forest_5_predict = forest_5.predict(test_data_v3)
forest_5_predict_int = (np.array(forest_5_predict)).astype('int')
forest_5_performance =  rmsle(test_labels_v3_np,forest_5_predict_int)
print('RMSLE with random forest w dummies ,time on market and period ind:', forest_5_performance)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSLE with random forest w dummies ,time on market and period ind: 0.313913282025


## Two model approach - Independent models for Casual vs. Registered

In [103]:
data_4 = raw_data
data_4 = feature_engineering_v3(data_4,market_launch)
train_data_v4,test_data_v4,train_labels_v4 ,test_labels_v4 = train_test_split(data_4, labels, test_size=0.20, random_state=42)

In [104]:
test_labels_v4_np = np.array(test_labels_v4['count'])

train_labels_v4_reg = train_data_v4[regress_registered]
train_labels_v4_cas = train_data_v4[regress_casual]
data_4 = keep_non_target_cols_only(data_4)
train_data_v4 = keep_non_target_cols_only(train_data_v4)
test_data_v4 = keep_non_target_cols_only(test_data_v4)

**Registered users model**

In [105]:
forest_reg = RandomForestRegressor(n_estimators = 1000,n_jobs = -1)
forest_6_reg = forest_reg.fit(train_data_v4,train_labels_v4_reg)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



**Casual Users**

In [106]:
forest_cas = RandomForestRegressor(n_estimators = 1000,n_jobs = -1)
forest_6_cas = forest_cas.fit(train_data_v4,train_labels_v4_cas)

C:\Users\romulo\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



**Predicting total** by summing predictions of individual models

In [111]:
forest_6_predict_reg = forest_6_reg.predict(test_data_v4)
forest_6_predict_cas = forest_6_cas.predict(test_data_v4)
#Adding up
forest_6_predict = forest_6_predict_reg + forest_6_predict_cas
forest_6_predict_int  = (np.array(forest_6_predict)).astype('int')

**Results:**

In [112]:
forest_6_performance =  rmsle(test_labels_v4_np,forest_6_predict_int)
print('RMSLE with random forest composite:', forest_6_performance)

RMSLE with random forest composite: 0.312292352006


Minor improvement of roughly 0.01